In [1]:
import tensorflow as tf
import numpy as np
from random import shuffle 

In [2]:
# Generate training data
train_input = ['{0:020b}'.format(i) for i in range(2**20)]
shuffle(train_input)
train_input = [map(int,i) for i in train_input]
ti  = []
for i in train_input:
    temp_list = []
    for j in i:
            temp_list.append([j])
    ti.append(np.array(temp_list))
train_input = ti

In [3]:
# convert to one hot
train_output = []

for i in train_input:
    count = 0
    for j in i:
        if j[0] == 1:
            count+=1
    temp_list = ([0]*21)
    temp_list[count]=1
    train_output.append(temp_list)

In [4]:
# Generate test data
NUM_EXAMPLES = 10000
test_input = train_input[NUM_EXAMPLES:] 
test_output = train_output[NUM_EXAMPLES:] #everything beyond 10,000
train_input = train_input[:NUM_EXAMPLES]
train_output = train_output[:NUM_EXAMPLES] #till 10,000

In [5]:
# Deep learning code now
data = tf.placeholder(tf.float32, [None, 20,1]) 
target = tf.placeholder(tf.float32, [None, 21])

In [6]:
# Recurrent NN
num_hidden = 24
cell = tf.nn.rnn_cell.LSTMCell(num_hidden, state_is_tuple=True)

In [7]:
val, state = tf.nn.dynamic_rnn(cell, data, dtype=tf.float32)

In [8]:
val = tf.transpose(val, [1, 0, 2])
last = tf.gather(val, int(val.get_shape()[0]) - 1)

In [9]:
weight = tf.Variable(tf.truncated_normal([num_hidden, int(target.get_shape()[1])]))
bias = tf.Variable(tf.constant(0.1, shape=[target.get_shape()[1]]))

In [10]:
prediction = tf.nn.softmax(tf.matmul(last, weight) + bias)

In [11]:
cross_entropy = -tf.reduce_sum(target * tf.log(tf.clip_by_value(prediction,1e-10,1.0)))

In [12]:
optimizer = tf.train.AdamOptimizer()
minimize = optimizer.minimize(cross_entropy)

/Users/aditya/anaconda3/envs/hw3_1/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [13]:
mistakes = tf.not_equal(tf.argmax(target, 1), tf.argmax(prediction, 1))
error = tf.reduce_mean(tf.cast(mistakes, tf.float32))

In [14]:
init_op = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init_op)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [17]:
batch_size = 1000
no_of_batches = int(len(train_input)/batch_size)
epoch = 50
for i in range(epoch):
    ptr = 0
    for j in range(no_of_batches):
        inp, out = train_input[ptr:ptr+batch_size], train_output[ptr:ptr+batch_size]
        ptr+=batch_size
        sess.run(minimize,{data: inp, target: out})
    print("Epoch - ",str(i))
incorrect = sess.run(error,{data: test_input, target: test_output})
print('Epoch {:2d} error {:3.1f}%'.format(i + 1, 100 * incorrect))
sess.close()

Epoch -  0
Epoch -  1
Epoch -  2
Epoch -  3
Epoch -  4
Epoch -  5
Epoch -  6
Epoch -  7
Epoch -  8
Epoch -  9
Epoch -  10
Epoch -  11
Epoch -  12
Epoch -  13
Epoch -  14
Epoch -  15
Epoch -  16
Epoch -  17
Epoch -  18
Epoch -  19
Epoch -  20
Epoch -  21
Epoch -  22
Epoch -  23
Epoch -  24
Epoch -  25
Epoch -  26
Epoch -  27
Epoch -  28
Epoch -  29
Epoch -  30
Epoch -  31
Epoch -  32
Epoch -  33
Epoch -  34
Epoch -  35
Epoch -  36
Epoch -  37
Epoch -  38
Epoch -  39
Epoch -  40
Epoch -  41
Epoch -  42
Epoch -  43
Epoch -  44
Epoch -  45
Epoch -  46
Epoch -  47
Epoch -  48
Epoch -  49
Epoch 50 error 9.1%


In [19]:
print(sess.run(model.prediction,{data: [[[1],[0],[0],[1],[1],[0],[1],[1],[1],[0],[1],[0],[0],[1],[1],[0],[1],[1],[1],[0]]]}))

NameError: name 'model' is not defined

In [ ]:
# 